In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16418483
paper_name = 'powers_fields_2006' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/PowersRankedDeletions.xlsx', sheet_name='Sheet1', skiprows=3)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4759 x 11


In [10]:
original_data.head()

,rank,ORF,GENE,Putative TOR pathway,1 week,2 week,5 week,7 weeks,Description,Function,Localization
0,1,YCL025C,AGP1,*,1.940734,9.676680,8.265812,245.600440,amino acid transport,amino acid transporter activity,plasma membrane
1,2,YJL129C,TRK1,NaN,1.570524,7.315183,7.767504,190.225805,potassium ion homeostasis,potassium ion transporter activity,plasma membrane
2,3,YIL094C,LYS12,*,1.801833,9.420834,6.906460,145.066731,lysine biosynthesis,isocitrate dehydrogenase activity,cytoplasm*
3,4,YDR019C,GCV1,NaN,1.291409,1.535742,0.089664,107.189594,one-carbon compound metabolism*,glycine dehydrogenase (decarboxylating) activity,mitochondrion
4,5,YER091C-A,NaN,NaN,1.234110,3.728811,4.852029,95.751300,NaN,NaN,NaN


In [11]:
original_data['orf'] = original_data['ORF'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [rank, ORF, GENE, Putative TOR pathway, 1 week , 2 week , 5 week , 7 weeks, Description, Function, Localization, orf]
Index: []


In [15]:
original_data.set_index('orf', inplace=True)

In [19]:
original_data = original_data[['1 week ','2 week ','5 week ','7 weeks']].apply(pd.to_numeric, axis=1, errors='coerce')

In [20]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data.shape

(4691, 4)

# Prepare the final dataset

In [22]:
data = original_data.copy()

In [23]:
dataset_ids = [1306, 4778, 4779, 4780]
datasets = datasets.reindex(index=dataset_ids)

In [24]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [25]:
data.head()

dataset_id,1306,4778,4779,4780
data_type,value,value,value,value
orf,,,,
YAL002W,0.733743,0.190263,0.021093,-0.258453
YAL004W,1.177173,0.364029,0.311690,-0.413720
YAL005C,0.601160,0.300251,0.198385,-0.531488
YAL007C,0.984847,0.827014,0.620147,-0.116105
YAL008W,1.060685,0.891566,0.709733,-0.108947


## Subset to the genes currently in SGD

In [26]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [27]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,1306,4778,4779,4780
,data_type,value,value,value,value
gene_id,orf,,,,
2,YAL002W,0.733743,0.190263,0.021093,-0.258453
1863,YAL004W,1.177173,0.364029,0.311690,-0.413720
4,YAL005C,0.601160,0.300251,0.198385,-0.531488
5,YAL007C,0.984847,0.827014,0.620147,-0.116105
6,YAL008W,1.060685,0.891566,0.709733,-0.108947


# Normalize

In [28]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [29]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [30]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,1306,4778,4779,4780,1306,4778,4779,4780
,data_type,value,value,value,value,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,
2,YAL002W,0.733743,0.190263,0.021093,-0.258453,-0.479132,-0.783524,-0.139871,-0.099131
1863,YAL004W,1.177173,0.364029,0.311690,-0.413720,0.186255,-0.600020,0.047399,-0.125364
4,YAL005C,0.601160,0.300251,0.198385,-0.531488,-0.678080,-0.667372,-0.025619,-0.145262
5,YAL007C,0.984847,0.827014,0.620147,-0.116105,-0.102339,-0.111088,0.246178,-0.075081
6,YAL008W,1.060685,0.891566,0.709733,-0.108947,0.011460,-0.042919,0.303910,-0.073871


# Print out

In [31]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [32]:
from IO.save_data_to_db3 import *

In [33]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/4 [00:00<?, ?it/s]

Deleting all datasets for PMID 16418483...
Inserting the new data...


100%|██████████| 4/4 [00:28<00:00,  7.08s/it]

Updating the data_modified_on field...
